# Búsqueda de múltiples secuencias

### Autora: Lucía Núñez Calvo

#### Fecha: 4 de Abril de 2022

El objetivo de este cuaderno es mostrar como se pueden pueden localizar varias subsecuencias a partir de un único patrón de referencia dentro de una secuencia de mayor tamaño. Esto se usará más adelante para localizar 
todas las secuencias que comparantan similitudes con el patrón de referencia y acabar selecciónando la más parecida al mismo. 

En este caso la búsqueda se realizará con datos reales

In [1]:
import pickle
import numpy as np

def extrat_pickle(file):
    pos=[]
    with open("/.Pickle/"+file, "rb") as f:
        while True: 
            try:
                current_id=pickle.load(f)
                pos.append(current_id)
            except EOFError:
                break
    return pos

In [2]:
ang_ej1 = extrat_pickle("angulos_ej1_MultiplesSecuencias.pickle")
ang_paciente = extrat_pickle("angulos_total_MultiplesSecuencias.pickle")

print("Número de frames del ejercicio 1: ",len(ang_ej1))
print("Número de frames del ejercicio completo: ",len(ang_paciente))

Número de frames del ejercicio 1:  969
Número de frames del ejercicio completo:  2896


Una vez se han obtenido las secuencias relativas al ejercicio concreto a analizar y por otra parte la secuencia completa de ejercicios, el primer paso será crear las funciones necesárias para calcular:
- Matriz de distancias o matriz de costes locales C
- Matriz de costes acumulados D
- Ruta de deformación

In [16]:
from scipy.signal import find_peaks

from tslearn import metrics
from tslearn.generators import random_walks
from tslearn.preprocessing import TimeSeriesScalerMeanVariance


def calcule_matrix(X,Y):
    """
    Función que calcula la matriz de costes locales o matriz de distancias.
    Este cálculo se ha realizao mediante la distancia euclidiana.
    
    Parámetros de entrada 
    ---------------------
    X : Secuencia corta con un ejercicio concreto.
    Y : Secuencia larga con múltiples ejercicios.
    
    Salida
    ------
    Matriz de costes locales o de distancias.
    """
    C=np.zeros((len(X),len(Y)))
    for i in range(len(X)):
        for j in range(len(Y)):
            x=np.array(X[i])
            y=np.array(Y[j])
            
            #Cálculo de la distancia euclídea entre dos secuencias.
            C[i][j] = np.linalg.norm(np.array(X[i])-np.array(Y[j])) #para secuencias de varias dimensiones

    return C

def calcule_matrixD(C,X,Y):
    """
    Función que calcula la matriz de costes acumulados.
    Este cálculo se realizará de la siguiente forma:
        1) La primera fila de D está inicializada como D(1,m):=C(1,m) para m € [1:M] siendo M=len(Y).
        2) La primera columna de D está inicializada como D(n,1):=∑k=1nC(k,1) para n € [1:N] siendo N=len(X).
        3) El resto de los valores se definen recursivamente a partir de la matriz de costes locales.
    
    Parámetros de entrada 
    ---------------------
    C : Matriz de costes locales o de distancias.
    X : Secuencia corta con un ejercicio concreto.
    Y : Secuencia larga con múltiples ejercicios.
    
    Salida
    ------
    Matriz de costes acumulados.
    """
    D=np.zeros((len(X),len(Y)))
    D[:,0]=np.cumsum(C[:,0]) #inicialización de la primera columna
    D[0,:]=C[0,:] #Inicialización de la primera fila 

    for i in range(1,len(X)):
        for j in range(1,len(Y)):
            D[i][j] = C[i][j]+min(D[i-1][j],D[i][j-1],D[i-1][j-1])
            
    return D

def calcule_path(C,sz):
    """
    Función que calcula las posibles rutas de deformación.
    Estas rutas serán calculadas mediante la función scipy.signal.find_peaks que localiza picos dentro
    de una señal. Con el argumento "distance" se especifica distancia horizontal mínima entre picos, cuanto 
    más restrictivo sea este parámetro, menor será el número de caminos localizados.
    
    Parámetros de entrada 
    ---------------------
    C : Matriz de costes locales o de distancias.
    sz : Parámetro que se utiliza para hacer más o menos restrictiva la búsqueda.
    
    Salida
    ------
    Posibles rutas de deformación.
    """
    #Calculo del coste de la función
    cost_func = C[-1, :]

    #Identificación de los posibles caminos
    potential_matches = find_peaks(-cost_func, distance=sz * 2.75, height=-50)[0]

    #Calculo de las rutas óptimas a partir de cada uno de los mínimos identificados
    paths = [metrics.subsequence_path(C, match) for match in potential_matches]
    
    return paths

Una vez se tienen realizadas las funciones que realizan el cálculos de las distintas matrices y la ruta de deformación, el siguiene paso que se va a mostrar es probar los caminos que localiza a partir de diferentes valores restrictivos. 

In [30]:
#Medir el tiempo de ejecución
import time
inicio = time.time()

C=calcule_matrix(ang_ej1, ang_paciente)
#print("\nMatriz de distancias o matriz de costes locales con sqrt(x[i] - y[j])^2:\n ",C) 

valores_sz=[3,7,10,14,20,30,40,50,60,70]

for sz in valores_sz:
    paths=calcule_path(C,sz)
    print("Número de caminos óptimos para sz = "+str(sz)+": ",len(paths))
    
fin = time.time()
print("\nTiempo trascurrido en localizar los posibles caminos óptimos: ",fin-inicio, "segundos")

Número de caminos óptimos para sz = 3:  102
Número de caminos óptimos para sz = 7:  72
Número de caminos óptimos para sz = 10:  58
Número de caminos óptimos para sz = 14:  44
Número de caminos óptimos para sz = 20:  33
Número de caminos óptimos para sz = 30:  22
Número de caminos óptimos para sz = 40:  19
Número de caminos óptimos para sz = 50:  14
Número de caminos óptimos para sz = 60:  13
Número de caminos óptimos para sz = 70:  11

Tiempo trascurrido en localizar los posibles caminos óptimos:  32.60620164871216 segundos


<div class="alert alert-block alert-warning">
    <b>Nota: </b>Como se puede observar el tiempo de ejecución para calcular la matriz de distancias y encontrar un total de diez posibles caminos no es nada elevado.
    
Por otra parte también se puede ver como a medida que sube el valor del argumento que modifica la restricción en cuanto a la distancia horizontal entre picos, el número de caminos óptimos localizados disminuye. 
    
Con valores muy altos, como los primeros que se han obtenidos, es demasiado inviable realizar un posible cálculo ya que el número de caminos es elevado y por consiguiente muchos de ellos prácticamente iguales. Por esa razón se usarán restricciones más elevadas para sacar únicamente las secuencias que compartan un frado de similitud con el patrón de referencia lo suficientemente aceptable. 
</div>

<div class="alert alert-block alert-warning">
    <b>Nota: </b> Una vez obtenidos los diferentes caminos, se plantearán distintas métricas para su calsificaicón. Algunas de ellas pueden ser:

- Cálculo de la distancia DTW
- Cálculo del coste DTW
- Cálculo de la distancia euclidian
- Cálculo de la distancia manhattan
- Métrica Soft-DTW
    
    
En el cuaderno "C3_Búsqueda_y_agrupación_de_secuencias_SIMILARES.ipynb" se puede observar una de ellas.
</div>